In [ ]:
from flask import Flask, request, render_template
import pandas as pd
from autogluon.tabular import TabularPredictor

app = Flask(__name__)

# Load the pre-trained model
predictor = TabularPredictor.load('AutogluonModels/ag-20240429_230251/models/LightGBMXT/model.pkl')

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/predict', methods=['POST'])
def predict():
    # Retrieve input data from the form
    input_data = {feature: request.form[feature] for feature in request.form}
    input_df = pd.DataFrame([input_data])

    # Make prediction
    prediction = predictor.predict(input_df)
    probability = predictor.predict_proba(input_df)

    # Generate an alert based on the prediction
    if prediction.iloc[0] == 1:
        message = f'Flood Alert! Probability: {probability.iloc[0,1]:.2f}'
    else:
        message = f'No Flood. Probability: {probability.iloc[0,0]:.2f}'

    return render_template('index.html', prediction_text=message)

if __name__ == '__main__':
    app.run(debug=True)
